In [2]:
import struct
import numpy as np
import sys
from scipy.io import loadmat
from scipy.special import sph_harm
import autograd.numpy as np
from autograd import jacobian
import math
import matplotlib.pyplot as plt
from time import time
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [3]:
no_of_ver = 53215 #53490

In [4]:
fileName='Dataset/Coarse_Dataset/Exp_Pca.bin'
with open(fileName, mode='rb') as file: # b is important -> binary
#     fileContent = file.read()
    dim_exp = np.fromfile(file, dtype=np.int32, count=1)
    mu_exp = np.zeros(no_of_ver*3)
    base_exp = np.zeros((no_of_ver*3,dim_exp[0]), dtype=float)
    mu_exp = np.fromfile(file, dtype=float, count=3*no_of_ver)
    base_exp = np.fromfile(file, dtype=float, count=3*no_of_ver*dim_exp[0])

In [5]:
A_exp = np.array(np.resize(base_exp, (no_of_ver*3, dim_exp[0])))

In [6]:
data = np.loadtxt('Dataset/Coarse_Dataset/std_exp.txt', delimiter=' ')
data=data[:,np.newaxis]

In [7]:
temp = loadmat('Dataset/3DDFA_Release/Matlab/ModelGeneration/model_info.mat')
trimIndex = np.array(temp['trimIndex'][:,0], dtype=np.int32)
trim_ind = np.reshape(np.array([3*trimIndex-2,3*trimIndex-1,3*trimIndex])-1,(no_of_ver*3,),'F')#np.append(3*trimIndex-2,np.append( 3*trimIndex-1, 3*trimIndex))
tri_mesh_data = temp['tri'].T

In [8]:
lmks_3d_ind = temp['keypoints']

def load(path):
    """takes as input the path to a .pts and returns a list of 
	tuples of floats containing the points in in the form:
	[(x_0, y_0, z_0),
	 (x_1, y_1, z_1),
	 ...
	 (x_n, y_n, z_n)]"""
    with open(path) as f:
        rows = [rows.strip() for rows in f]
    
    """Use the curly braces to find the start and end of the point data""" 
    head = rows.index('{') + 1
    tail = rows.index('}')

    """Select the point data split into coordinates"""
    raw_points = rows[head:tail]
    coords_set = [point.split() for point in raw_points]

    """Convert entries from lists of strings to tuples of floats"""
    points = [tuple([float(point) for point in coords]) for coords in coords_set]
    return points

lmks_2d = np.array(load('Dataset/300W-Convert/300W-Original/afw/134212_1.pts'))

no_of_lmks = len(lmks_2d)
lmks = {}
lmks['2d'] = lmks_2d-[700,144]
lmks['3d'] = lmks_3d_ind

In [9]:
morph_model = loadmat('Dataset/PublicMM1/01_MorphableModel.mat')

In [10]:
morph_model.keys()

dict_keys(['__header__', '__version__', '__globals__', 'tl', 'shapeMU', 'shapePC', 'shapeEV', 'texMU', 'texPC', 'texEV', 'segbin', 'segMM', 'segMB'])

In [11]:
shapePCA = morph_model['shapePC']
shapeMU = morph_model['shapeMU']
shapeSTD = morph_model['shapeEV']

texPCA = morph_model['texPC']
texMU = morph_model['texMU']
texSTD = morph_model['texEV']

p_mu = shapeMU[trim_ind]
b_mu = texMU[trim_ind]
A_alb = np.array(texPCA[trim_ind,:100])
A_id = np.array(shapePCA[trim_ind,:100])
std_id = np.array(shapeSTD[:100])
std_alb = np.array(texSTD[:100])
std_exp = np.array(data)

In [12]:
PI = np.zeros((2,3))
PI[0,1] = 1
PI[1,0] = 1

In [13]:
chi = np.random.random_sample((312,1))

$$ \chi_g = \{\alpha_{id},\alpha_{exp}\} $$
$$ \chi_p = \{\textit{pitch}, \textit{yaw}, \textit{roll}\} $$
$$ \chi_t = \{s, \textbf{t}\} $$
$$ \chi_l = \{\alpha_{alb}, \textbf{r}\} $$

In [14]:
al_id = chi[0:100]
al_exp = chi[100:179]
al_alb = chi[179:279]
[s, pitch, yaw, roll] = chi[279:283,0]
t = chi[283:285]
r = chi[285:]
gamma_r = r[:9]
gamma_g = r[9:18]
gamma_b = r[18:]

$E(\chi) = E_{con} + w_lE_{lan} + w_rE_{reg}$ 

$$E(\chi) = \frac{1}{|F|} \|I_{ren} - I_{in}\|^2 + w_l*\frac{1}{|L|} \sum_{i\in L} \|\textbf{q}_i - (\Pi R\textbf{p}_i + \textbf{t})\|^2+ w_r*\sum_{i=1}^{100} [(\frac{\alpha_{id,i}}{\sigma_{id,i}})^2+(\frac{\alpha_{alb,i}}{\sigma_{alb,i}})^2] + \sum_{i=1}^{79} (\frac{\alpha_{exp,i}}{\sigma_{exp,i}})^2$$

In [15]:
def P(l, m, x):
    pmm = 1.0
    if m>0:
        somx2 = np.sqrt((1.0-x)*(1.0+x))
        fact = 1.0
        for i in range(m):
            pmm = -fact*pmm*somx2
            fact = fact+2.0
    if l==m :
        return pmm
    pmmp1 = x*(2.0*m+1.0)*pmm
    if (l==m+1):
        return pmmp1
    pll = 0.0
    for ll in range(m+2, l+1):
        pll = ((2.0*ll-1.0)*x*pmmp1 - (ll+m-1.0)*pmm)/(ll-m)
        pmm = pmmp1
        pmmp1 = pll
    return pll

def factorial(n):
    return np.prod(range(1,n+1))

def K(l, m):
    norm_const = ((2.0*l+1.0)*factorial(l-m))/((4.0*np.pi)*factorial(l+m))
    return np.sqrt(norm_const)

def SH(m, l, phi, theta):
    sqrt2 = np.sqrt(2.0)
    if m==0:
        return K(l,0)*P(l,m,np.cos(theta))
    elif m>0:
        return sqrt2*K(l,m)*np.cos(m*phi)*P(l,m,np.cos(theta))
    else:
        return sqrt2*K(l,-m)*np.sin(-m*phi)*P(l,-m,np.cos(theta))

In [16]:
def rot_mat(pitch, yaw, roll):
    Rx = np.array([[1,0,0],
                   [0,np.cos(roll),-np.sin(roll)],
                   [0,np.sin(roll),np.cos(roll)]])
    Ry = np.array([[np.cos(pitch),0,np.sin(pitch)],
                   [0,1,0],
                   [-np.sin(pitch),0,np.cos(pitch)]])
    Rz = np.array([[np.cos(yaw),-np.sin(yaw),0],
                   [np.sin(yaw),np.cos(yaw),0],
                   [0,0,1]])
    R = Rz@Ry@Rx
    return R

def sh_basis(n):
    theta = n[1] #Polar angle
    phi = n[0] #Azimuth angle
    sh = np.zeros((9,), dtype=np.float32)
    count = 0
    
#     for l in range(3):
#         for m in np.arange(-l,l+1):
#             temp = sph_harm(m,l,phi,theta)
#             if m==0:
#                 sh[count]=np.real(temp)
#             elif m>0:
#                 sh[count]=np.sqrt(2)*np.real(temp)
#             else:
#                 sh[count]=np.sqrt(2)*np.imag(temp)
#             count = count+1
#             sh[count] = SH(m,l,phi,theta)
#             count=count+1
        
    
    return np.array([SH(m,l,phi,theta) for l in range(3) for m in range(-l,l+1) ])

In [17]:
p = p_mu + np.matmul(A_id,al_id) + np.matmul(A_exp,al_exp)
b = b_mu + np.matmul(A_alb,al_alb)
vertex = np.reshape(p, (no_of_ver, 3))
albedo = np.reshape(b, (no_of_ver, 3))

In [18]:
"""My rasterization code...not used in the program"""
# def barycentric_weights(p, tri_p):
#     #http://blackpawn.com/texts/pointinpoly/
    
#     a = tri_p[0,:]
#     b = tri_p[1,:]
#     c = tri_p[2,:]
    
#     v0 = c - a
#     v1 = b - a
#     v2 = p - a
    
#     dot00 = np.dot(v0,v0)
#     dot01 = np.dot(v0,v1)
#     dot02 = np.dot(v0,v2)
#     dot11 = np.dot(v1,v1)
#     dot12 = np.dot(v1,v2)
    
#     denom = dot00*dot11 - dot01*dot01
    
#     if denom == 0:
#         u = v = 0
#     else:
#         u = (dot11*dot02-dot01*dot12)/denom
#         v = (dot00*dot12-dot01*dot02)/denom
        
#     weights = np.array([1-u-v, u, v])
        
#     return weights

# def rasterize(q, q_depth, tri_mesh_data, h, w):
#     depth_info = -math.inf*np.ones((h,w))
# #     depth_info = {}
#     tri_ind_info = -np.ones((h,w), dtype = np.int32)
#     bary_wts_info = {}
    
#     for i in range(h):
#         for j in range(w):
# #             depth_info[(i,j)] = -math.inf
#             bary_wts_info[(i,j)] = 0
    
#     for i in range(len(tri_mesh_data)):
#         print('rasterizing: ',i+1)
#         tri_ver_ind = tri_mesh_data[i,:]
        
#         umin = max(int(np.ceil(np.min(q[tri_ver_ind, 0]))), 0)
#         umax = min(int(np.floor(np.max(q[tri_ver_ind, 0]))), w-1)
        
#         vmin = max(int(np.ceil(np.min(q[tri_ver_ind, 1]))), 0)
#         vmax = min(int(np.floor(np.max(q[tri_ver_ind, 1]))), h-1)
        
#         if umax<umin or vmax<vmin:
#             continue
#         else:
#             for u in range(umin, umax+1):
#                 for v in range(vmin, vmax+1):
#                     weights = barycentric_weights([u, v], q[tri_ver_ind, :])
#                     if (weights<0).all():
#                         continue
#                     else:
#                         depth = np.dot(weights, q_depth[tri_ver_ind])
#                         if depth > depth_info[(u, v)]:
#                             depth_info[u, v] = depth
#                             tri_ind_info[u, v] = i
#                             bary_wts_info[(u, v)] = weights
                            
#     return tri_ind_info, bary_wts_info

def world_to_image(q_world, h, w):
    temp = np.array([w/2,h/2-h+1,0])
    q_image = (q_world + temp)*[1,-1,1]
    
#     q_image[:,0] = q_image[:,0] + w/2
#     q_image[:,1] = q_image[:,1] + h/2
#     q_image[:,1] = h - q_image[:,1] - 1
    
    return q_image

def cart2sph(n):
    temp = n[1]/n[0]
    
    if n[0]==0:
        if n[1]<0:
            phi = -np.pi/2
        else:
            phi = np.pi/2
    else:
        if n[0]>0:
            phi = np.arctan(temp)
        elif n[1]<0:
            phi = np.arctan(temp) - np.pi
        else:
            phi = np.arctan(temp) + np.pi
#     phi = np.arctan() #arctan(y/x)
    theta = np.arccos(n[2]) #arccos(z)
    
    return [phi, theta]

def calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w):
#     normal_xyz = np.zeros((h, w, 3))
#     normal_sph = np.zeros((h, w, 2))
    normal_xyz = {}
    normal_sph = {}
    
    for i in range(h):
        for j in range(w):
            normal_xyz[(i,j)] = 0
            normal_sph[(i,j)] = 0
            tri_ver = q[tri_mesh_data[tri_ind_info[i, j]-1, :],:]
            a = tri_ver[0,:]
            b = tri_ver[1,:]
            c = tri_ver[2,:]
            normal_xyz[(i,j)] = np.cross(a-b, b-c)/np.linalg.norm(np.cross(a-b, b-c))
            if np.dot(np.mean(tri_ver, 0)-centroid, normal_xyz[(i,j)])<0:
                normal_xyz[(i,j)] *= -1
            normal_sph[(i,j)] = cart2sph(normal_xyz[(i,j)])
    return normal_sph

def render_color_image(q, albedo, tri_mesh_data, gamma, h, w):
#     image = np.zeros((h,w,3), dtype=np.float32)
    centroid = np.mean(q,0)
    
#     tri_ind_info, bary_wts_info = rasterize(q[:,:2], q[:,2], tri_mesh_data, h, w)
    st = time()
    depth_info, tri_ind_info, bary_wts_info = rasterize_triangles(q, tri_mesh_data, h, w)
    print(time()-st)
    
    n_sph = calculate_normal(tri_ind_info, tri_mesh_data, centroid, q, h, w)
    
#     for i in range(h):
#         for j in range(w):
#             sh_func = sh_basis(n_sph[(i,j)])
#             image[i,j,:] = (albedo[tri_mesh_data[tri_ind_info[i, j], :],:].T@bary_wts_info[(i,j)])*(gamma.T@sh_func.squeeze())
#             image[i,j,:] = alb[i,j,:]*(gamma.T@sh_func.squeeze())
    image = np.array([[(albedo[tri_mesh_data[tri_ind_info[i, j], :],:].T@bary_wts_info[(i,j)])*(gamma.T@sh_basis((n_sph[(i,j)])).squeeze()) for j in range(w)] for i in range(h)])
    return image

In [19]:
def isPointInTri(point, tri_points):
    ''' Judge whether the point is in the triangle
    Method:
        http://blackpawn.com/texts/pointinpoly/
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        bool: true for in triangle
    '''
    tp = tri_points

    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]

    # dot products
    dot00 = np.dot(v0.T, v0)
    dot01 = np.dot(v0.T, v1)
    dot02 = np.dot(v0.T, v2)
    dot11 = np.dot(v1.T, v1)
    dot12 = np.dot(v1.T, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

    # check if point in triangle
    return (u >= 0) & (v >= 0) & (u + v < 1)

def get_point_weight(point, tri_points):
    ''' Get the weights of the position
    Methods: https://gamedev.stackexchange.com/questions/23743/whats-the-most-efficient-way-to-find-barycentric-coordinates
     -m1.compute the area of the triangles formed by embedding the point P inside the triangle
     -m2.Christer Ericson's book "Real-Time Collision Detection". faster.(used)
    Args:
        point: (2,). [u, v] or [x, y] 
        tri_points: (3 vertices, 2 coords). three vertices(2d points) of a triangle. 
    Returns:
        w0: weight of v0
        w1: weight of v1
        w2: weight of v3
     '''
    tp = tri_points
    # vectors
    v0 = tp[2,:] - tp[0,:]
    v1 = tp[1,:] - tp[0,:]
    v2 = point - tp[0,:]

    # dot products
    dot00 = np.dot(v0.T, v0)
    dot01 = np.dot(v0.T, v1)
    dot02 = np.dot(v0.T, v2)
    dot11 = np.dot(v1.T, v1)
    dot12 = np.dot(v1.T, v2)

    # barycentric coordinates
    if dot00*dot11 - dot01*dot01 == 0:
        inverDeno = 0
    else:
        inverDeno = 1/(dot00*dot11 - dot01*dot01)

    u = (dot11*dot02 - dot01*dot12)*inverDeno
    v = (dot00*dot12 - dot01*dot02)*inverDeno

    w0 = 1 - u - v
    w1 = v
    w2 = u

    return w0, w1, w2

def rasterize_triangles(vertices, triangles, h, w):
    ''' 
    Args:
        vertices: [nver, 3]
        triangles: [ntri, 3]
        h: height
        w: width
    Returns:
        depth_buffer: [h, w] saves the depth, here, the bigger the z, the fronter the point.
        triangle_buffer: [h, w] saves the tri id(-1 for no triangle). 
        barycentric_weight: [h, w, 3] saves corresponding barycentric weight.

    # Each triangle has 3 vertices & Each vertex has 3 coordinates x, y, z.
    # h, w is the size of rendering
    '''
    # initial 
    depth_buffer = {}#np.zeros([h, w]) - 999999. #+ np.min(vertices[2,:]) - 999999. # set the initial z to the farest position
    triangle_buffer = np.zeros([h, w], dtype = np.int32) - 1  # if tri id = -1, the pixel has no triangle correspondance
    barycentric_weight = {}#np.zeros([h, w, 3], dtype = np.float32)  # 
    
    for i in range(h):
        for j in range(w):
            depth_buffer[(i,j)] = -math.inf
            barycentric_weight[(i,j)] = np.array([0, 0, 0])
    
    for i in range(triangles.shape[0]):
#         print('Rasterzing: ',i+1)
        tri = triangles[i, :] # 3 vertex indices

        # the inner bounding box
        umin = max(int(np.ceil(np.min(vertices[tri, 0]))), 0)
        umax = min(int(np.floor(np.max(vertices[tri, 0]))), w-1)

        vmin = max(int(np.ceil(np.min(vertices[tri, 1]))), 0)
        vmax = min(int(np.floor(np.max(vertices[tri, 1]))), h-1)

        if umax<umin or vmax<vmin:
            continue

        for u in range(umin, umax+1):
            for v in range(vmin, vmax+1):
                if not isPointInTri([u,v], vertices[tri, :2]): 
                    continue
                w0, w1, w2 = get_point_weight([u, v], vertices[tri, :2]) # barycentric weight
                point_depth = w0*vertices[tri[0], 2] + w1*vertices[tri[1], 2] + w2*vertices[tri[2], 2]
                if point_depth > depth_buffer[v, u]:
                    depth_buffer[(v, u)] = point_depth
                    triangle_buffer[v, u] = i
                    barycentric_weight[(v, u)] = np.array([w0, w1, w2])

    return depth_buffer, triangle_buffer, barycentric_weight

In [20]:
I_in = plt.imread('Dataset/300W-Convert/300W-Original/afw/134212_1.jpg')
I_in=I_in[144:400,700:956,:]#I_in[160:384,704:928,:]
h = w = 256

In [21]:
polygon = Polygon(lmks['2d'][:17,:])
temp2 = np.empty((h,w))
for i in range(h):
    for j in range(w):
        point = Point(i,j)
        temp2[i,j] = polygon.contains(point)
no_of_face_pxls = np.sum(temp2==1)

In [21]:
def E(chi, I_in, A_id, A_exp, A_alb, p_mu, b_mu, tri_mesh_data, lmks):
    al_id = chi[0:100]
    al_exp = chi[100:179]
    al_alb = chi[179:279]
    [s, pitch, yaw, roll] = chi[279:283,0]
    t = chi[283:286]
    r = chi[286:]
    gamma = np.reshape(r,(9,3),'F')
    lmks_2d = lmks['2d']
    lmks_3d_ind = lmks['3d']
    
    R = rot_mat(pitch, yaw, roll)
    
    p = p_mu + A_id@al_id + A_exp@al_exp
    b = b_mu + A_alb@al_alb
    vertex = np.reshape(p, (no_of_ver, 3))
    albedo = np.reshape(b, (no_of_ver, 3))
    
    q_world = s*R@vertex.T+t
#     q_depth = [0, 0, 1]@R@vertex.T
    
    q_image = world_to_image(q_world.T, h, w)
#     tri_ind_info, bary_wts_info = rasterize_triangles(q_image, tri_mesh_data, h, w)
#     return tri_ind_info,bary_wts_info,albedo
    
    I_rend = render_color_image(q_image, albedo, tri_mesh_data, gamma, h, w)
    
    w_l = 10
    w_r = 5e-5
    E_con = (1/no_of_face_pxls)*np.linalg.norm(I_rend - I_in)**2 #No of face pixels is apporximately 28241
    E_lan = (1/no_of_lmks)*np.linalg.norm(lmks_2d - q_image[lmks_3d_ind[0,:],:2])**2 #68 landmarks
    E_reg = np.linalg.norm(al_id/std_id)**2 + np.linalg.norm(al_alb/std_alb)**2 + np.linalg.norm(al_exp/std_exp)**2
    return E_con + w_l*E_lan + w_r*E_reg

In [24]:
jacobian_E = jacobian(E)
chi = np.random.rand(313,1)
# jacobian_E(chi, I_rend, I_in, q, points, R, PI)
chi[0:100] = chi_data[0:100]
chi[100:179] = chi_data[100:179]
chi[280:283,0] = chi_data[179:182,0]
chi[283:285] = chi_data[182:184]
chi[279,0] = chi_data[184,0]
chi_prev = chi
count = 1

while True:
#     chi_prev[279] = 100/(np.max(vertex) - np.min(vertex))
    chi_prev[0:100] = chi_data[0:100]
    chi_prev[100:179] = chi_data[100:179]
    chi_prev[280:283,0] = chi_data[179:182,0]
    chi_prev[283:285] = chi_data[182:184]
    chi_prev[279,0] = chi_data[184,0]
    print("Iteration No: ", count)
    if count==1:
#         chi_prev[279,0] = 200/(np.max(vertex) - np.min(vertex))
        E_val = E(chi_prev, I_in, A_id, A_exp, A_alb, p_mu, b_mu, tri_mesh_data-1, lmks)
    print(chi_prev[279,0])
    J = jacobian_E(chi_prev, I_in, A_id, A_exp, A_alb, p_mu, b_mu, tri_mesh_data-1, lmks)
    chi_next = chi_prev - np.linalg.pinv(J@J.T)@J*E_val
    E_val = E(chi_next, I_in, A_id, A_exp, A_alb, p_mu, b_mu, tri_mesh_data-1, lmks)#np.linalg.norm(chi_next - chi_prev.T)
    chi_prev = chi_next
    count=count+1
    print('Error: ',E_val)
    if E_val<10:
        break

Iteration No:  1
13.128201484680176
0.00118714
33.35515856742859
26.17777419090271
Error:  69494483.22211605
Iteration No:  2
0.00118714
33.112648725509644
24.75240683555603
Error:  185295819717.9219
Iteration No:  3
0.00118714
33.212615966796875
23.910141229629517
Error:  2.7710033463680077e+18
Iteration No:  4
0.00118714
32.3837525844574
27.073165893554688
Error:  7.418335168348677e+32
Iteration No:  5
0.00118714
33.08201289176941
789.5490894317627
Error:  5.778474172868881e+80
Iteration No:  6
0.00118714
33.404380798339844


KeyboardInterrupt: 

In [ ]:
al_id = chi_prev[0:100]
al_exp = chi_prev[100:179]
al_alb = chi_prev[179:279]
[s, pitch, yaw, roll] = chi_prev[279:283,0]
t = chi_prev[283:286]
r = chi_prev[286:]
gamma = np.reshape(r,(9,3),'F')

R = rot_mat(pitch, yaw, roll)

p = p_mu + np.matmul(A_id,al_id) + np.matmul(A_exp,al_exp)
b = b_mu + np.matmul(A_alb,al_alb)
vertex = np.reshape(p, (no_of_ver, 3))
albedo = np.reshape(b, (no_of_ver, 3))

q_world = s*R@vertex.T+t
#     q_depth = [0, 0, 1]@R@vertex.T

q_image = world_to_image(q_world.T, h, w)
I_rend = render_color_image(q_image, albedo, tri_mesh_data-1, gamma, h, w)

In [ ]:
im = I_rend+np.abs(np.min(I_rend))
im = im/np.max(im)
plt.imshow(im)

In [ ]:
fig = plt.figure()

ax = plt.axes(projection='3d')
ax.scatter3D(vertex[:,0],vertex[:,1],vertex[:,2])

In [ ]:
plt.imshow(I_rend)

In [23]:
chi_data = np.loadtxt('test_chi.txt')
chi_data = chi_data[:,np.newaxis]

In [ ]:
chi_data.shape

In [ ]:
chi[280:283,0].shape #= chi_data[179:182,:]

In [ ]:
np.random.rand(313,1)

In [37]:
sh_basis([20,20])

array([ 0.28209479, -0.40723506,  0.19938992, -0.18203208,  0.33925199,
       -0.37160168, -0.15782418, -0.16610413, -0.30365866])

In [36]:
SH(0,0,0,20)

0.28209479177387814